# Python for (open) Neuroscience

_Lecture 2.1_ - Real world Python on real world data 

Luigi Petrucco

Jean-Charles Mariani

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/lectures/Lecture2.0_Real-world-Python.ipynb)

## Working with images

Two
- 2D, single channel image: (n, m) matrix of pixels
- 2D RGB image: (n, m, 3) matrix where the third dimension represents values over each of three color axes (Red, Green, Blue)
- 3D, single channel image: ()

(End of lecture - code below to create the example data structure)

In [ ]:
data_folder = Path("/Users/vigji/sample_data_folder")
path_to_move.mkdir(exist_ok=True)

for subject_i in range(3):
    for session_i in range(4):
        with open(data_folder / f"sub{subject_i}_ses{session_i}.data", "w") as f:
            f.write("a")
        with open(data_folder / f"sub{subject_i}_ses{session_i}.metadata", "w") as f:
            f.write("a")